# Deepseek对生成内容评分

### 库和函数

In [2]:
from typing import List
from typing import Optional
from typing import Dict

import os
import json
import jsonlines
from pathlib import Path

import httpx
from openai import OpenAI

import requests

import time
import re
from tqdm import tqdm
import random

In [ ]:
with open('../prompts/Prompts/answer_zh.txt', 'r', encoding='utf-8') as f:
    content = f.read()

In [4]:
new_data = []

def transform_jsonl(input_file):
    result_list = []
    
    # 定义用于分割的标记
    user_header = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n"
    assistant_header = "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
    
    with open(input_file, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            prompt = data["prompt"]
            
            # 提取instruction部分
            instruction_start = prompt.find(user_header) + len(user_header)
            instruction_end = prompt.find("\n", instruction_start)
            instruction = prompt[instruction_start:instruction_end]
            
            # 提取input部分
            input_start = instruction_end + 1
            input_end = prompt.find(assistant_header, input_start)
            input_text = prompt[input_start:input_end]
            
            # 构建新字典
            transformed = {
                "instruction": instruction,
                "input": input_text,
                "answer": data["predict"],
                "output": data["label"]
            }
            result_list.append(transformed)
    
    return result_list

input_file = "./generated_predictions.jsonl"  # 替换为实际文件名
transformed_data = transform_jsonl(input_file)

# 打印前3个转换结果示例
for i, item in enumerate(transformed_data[:3]):
    print(f"Item {i+1}:")
    print(json.dumps(item, ensure_ascii=False, indent=2))
    print("-" * 50)

sample_100 = random.sample(transformed_data, 100)

Item 1:
{
  "instruction": "，负责组织应急加密和移动观测及资料传输；3.负责组织气象装备的运行、监控、维修和调配；4.按要求参加天气会商和应急工作例会;5.完成领导小组交办的其他任务。",
  "input": "Assistant",
  "answer": "观测司的职责包括负责组织各类气象观测资料的收集、传输和处理，根据预报司需求组织应急加密和移动观测及资料传输，负责组织气象装备的运行、监控、维修和调配，按要求参加天气会商和应急工作例会，完成领导小组交办的其他任务。",
  "output": "观测司负责组织各类气象观测资料的收集、传输和处理；根据预报司需求，负责组织应急加密和移动观测及资料传输；负责组织气象装备的运行、监控、维修和调配；按要求参加天气会商和应急工作例会；完成领导小组交办的其他任务。\n"
}
--------------------------------------------------
Item 2:
{
  "instruction": "故灾难等级和分级响应原则，由相应的地方人民政府组成现场应急救援指挥部，总指挥由地方政府负责人担任，全面负责应急救援指挥工作。按照有关规定由熟悉事故现场情况的有关领导具体负责现场救援指挥。现场应急救援指挥部及时向安全监管总局报告事故及救援情况，需要外部力量增援的，报请安全监管总局协调，并说明需要的救援力量、救援装备等情况。",
  "input": "Assistant",
  "answer": "现场应急救援指挥部及时向安全监管总局报告事故及救援情况，需要外部力量增援的，报请安全监管总局协调，并说明需要的救援力量、救援装备等情况。",
  "output": "该指挥部将向安全监管总局报告事故及救援情况，并表明自己不具有足够的救援能力，同时请求医疗队伍和重型机械的增援。\n"
}
--------------------------------------------------
Item 3:
{
  "instruction": "",
  "input": "突发公共卫生事件的分级反应：特别重大突发公共卫生事件应急处理工作由国务院或国务院卫生行政部门和有关部门组织实施，开展突发公共卫生事件的医疗卫生应急、信息发布、宣传教育、科研攻关、国际交流与合作、应急物资与设

In [ ]:
client = OpenAI(api_key="xxx", base_url="https://api.deepseek.com")

In [ ]:
answer = sample_100  # 使用随机抽样的100条数据

for idx, value in tqdm(enumerate(answer), total=len(answer), desc="Processing"):
    # 使用缓存内容并添加问题
    context = json.dumps(value, ensure_ascii=False, indent=2)
    question = content.replace("[[CONTEXT]]", context)
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": question},
        ],
        stream=False
    )
    # 构建 judgement 字典
    generation = response.choices[0].message.content
    judgement = {
        "id": idx,
        "rule": value,
        "generation": generation,
    }
    # 打开文件以进行写入，如果文件不存在，会创建文件
    with jsonlines.open('xxx', mode='a') as writer:
        writer.write(judgement)

Processing: 100%|██████████| 100/100 [25:47<00:00, 15.48s/it]
